In [151]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.dummy import DummyClassifier
from sklearn.model_selection import cross_val_score, GridSearchCV, ParameterGrid, train_test_split
from sklearn import metrics

import pandas as pd
import numpy as np

from sklearn.neighbors import KNeighborsClassifier

# # import warnings filter
# from warnings import simplefilter
# # ignore all future warnings
# simplefilter(action='ignore', category=FutureWarning)

In [152]:
# dataset_path = 'allUsers.lcl.csv'
dataset_path = "../data/tommaso_data/allUsers.lcl.csv"

df = pd.read_csv(dataset_path)
#knn Cannot work with mining data, so replace them with a 0
df = df.replace('?', 0)
#df = df.dropna(thresh=len(df) * 0.10, axis=1)


In [153]:
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, precision_score, recall_score

def save_results(clf, X_test, y_test, nclf, dataf):
    dataf.loc[nclf, 'acc'] = accuracy_score(y_test, clf.predict(X_test))
    dataf.loc[nclf,'prec'] = precision_score(y_test, clf.predict(X_test), average='weighted')
    dataf.loc[nclf,'recall'] = recall_score(y_test, clf.predict(X_test), average='weighted')
    dataf.loc[nclf, 'test f1 score (W)'] = f1_score(y_test, clf.predict(X_test), average='macro')
    return dataf

results_df = pd.DataFrame()



# KNN

# Class recognition with User label

In [154]:
df2 = df.drop(['Class'], axis=1)
labels_Class = df['Class']

X_train, X_test, y_train, y_test,  = train_test_split(df2, labels_Class, test_size=0.20, random_state=42)


In [155]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=1)
knn.fit(X_train, y_train)


KNeighborsClassifier(n_neighbors=1)

In [156]:
print(knn.score(X_test, y_test))


0.9602432778489116


In [157]:
results_df = save_results(knn, X_test, y_test, 'KNN', results_df)

# Hand owner recognition with Class label

In [158]:
df2 = df.drop(['User'], axis=1)
labels_Users = df['User']
X_train, X_test, y_train, y_test,  = train_test_split(df2, labels_Users, test_size=0.20, random_state=42)

In [159]:
knn = KNeighborsClassifier(n_neighbors=1)
knn.fit(X_train, y_train)


KNeighborsClassifier(n_neighbors=1)

In [160]:
print(knn.score(X_test, y_test))

0.8646606914212548


In [161]:
results_df = save_results(knn, X_test, y_test, 'KNN', results_df)


# Class recognition without User label

In [162]:
df2 = df.drop(['User'], axis=1)
df2 = df2.drop(['Class'], axis=1)
labels_Class = df['Class']
X_train, X_test, y_train, y_test,  = train_test_split(df2, labels_Class, test_size=0.20, random_state=42)

In [163]:
knn = KNeighborsClassifier(n_neighbors=1)
knn.fit(X_train, y_train)
print(knn.score(X_test, y_test))

0.9596670934699104


In [164]:
results_df = save_results(knn, X_test, y_test, 'KNN', results_df)


# Class recognition without Class label

In [165]:
df2 = df.drop(['User'], axis=1)
df2 = df2.drop(['Class'], axis=1)

labels_Users = df['User']
X_train, X_test, y_train, y_test,  = train_test_split(df2, labels_Users, test_size=0.20, random_state=42)

In [166]:
knn = KNeighborsClassifier(n_neighbors=1)
knn.fit(X_train, y_train)
print(knn.score(X_test, y_test))

0.8646606914212548


In [167]:
results_df = save_results(knn, X_test, y_test, 'KNN', results_df)


# SVM

In [168]:
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler

import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
# !pip insall sklearn.grid_search
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix

# dataset_path = 'allUsers.lcl.csv'

df = pd.read_csv(dataset_path)

df = df.replace('?', np.nan)
df = df.dropna(thresh=len(df) * 0.8 , axis=1)
df = df.replace(np.nan, 0)

drop_indices = np.random.choice(df.index, 75000, replace=False)
df = df.drop(drop_indices)

# Class recognition without User label

In [169]:
df2 = df.drop(['User'], axis=1)
df2 = df2.drop(['Class'], axis=1)
labels_Class = df['Class']
X_train, X_test, y_train, y_test,  = train_test_split(df2, labels_Class, test_size=0.20, random_state=42)

clf = SVC(kernel='rbf', gamma="auto", C = 7)
clf.fit(X_train, y_train)
print(clf.score(X_test, y_test))

0.2564516129032258


In [170]:
results_df = save_results(clf, X_test, y_test, 'SVC', results_df)


C:\Users\polcg\.virtualenvs\MD2-NovaiZ3s\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# SVC gread search class recognition without user label

In [171]:
param_grid = {'C':[1,10,100,1000],
              'gamma':[1,0.1,0.001,0.0001], 
              'kernel':['sigmoid','rbf']}

grid = GridSearchCV(SVC(),param_grid,refit = True, verbose=2)

grid.fit(X_train,y_train)


Fitting 5 folds for each of 32 candidates, totalling 160 fits
[CV] END .......................C=1, gamma=1, kernel=sigmoid; total time=   0.1s
[CV] END .......................C=1, gamma=1, kernel=sigmoid; total time=   0.1s
[CV] END .......................C=1, gamma=1, kernel=sigmoid; total time=   0.1s
[CV] END .......................C=1, gamma=1, kernel=sigmoid; total time=   0.1s
[CV] END .......................C=1, gamma=1, kernel=sigmoid; total time=   0.1s
[CV] END ...........................C=1, gamma=1, kernel=rbf; total time=   0.6s
[CV] END ...........................C=1, gamma=1, kernel=rbf; total time=   0.6s
[CV] END ...........................C=1, gamma=1, kernel=rbf; total time=   0.7s
[CV] END ...........................C=1, gamma=1, kernel=rbf; total time=   0.6s
[CV] END ...........................C=1, gamma=1, kernel=rbf; total time=   0.6s
[CV] END .....................C=1, gamma=0.1, kernel=sigmoid; total time=   0.1s
[CV] END .....................C=1, gamma=0.1, k

GridSearchCV(estimator=SVC(),
             param_grid={'C': [1, 10, 100, 1000],
                         'gamma': [1, 0.1, 0.001, 0.0001],
                         'kernel': ['sigmoid', 'rbf']},
             verbose=2)

In [172]:
grid.best_params_
predic = grid.predict(X_test)
print(grid.score(X_test, y_test))
print(grid.best_params_)

0.8467741935483871
{'C': 10, 'gamma': 0.0001, 'kernel': 'rbf'}


In [173]:
results_df = save_results(grid, X_test, y_test, 'SVC', results_df)


# Hand owner recognition without Class label


In [174]:
df2 = df.drop(['User'], axis=1)
df2 = df2.drop(['Class'], axis=1)
labels_Users = df['User']
X_train, X_test, y_train, y_test,  = train_test_split(df2, labels_Users, test_size=0.20, random_state=42)

clf = SVC(kernel='rbf', gamma="auto", C = 7)
clf.fit(X_train, y_train)
print(clf.score(X_test, y_test))

0.17096774193548386


In [175]:
results_df = save_results(clf, X_test, y_test, 'SVC', results_df)


C:\Users\polcg\.virtualenvs\MD2-NovaiZ3s\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# Gread SearchHand owner recognition without Class label

In [176]:
param_grid = {'C':[1,10,100,1000],
              'gamma':[1,0.1,0.001,0.0001], 
              'kernel':['sigmoid','rbf']}

grid = GridSearchCV(SVC(),param_grid,refit = True, verbose=2)

grid.fit(X_train,y_train)

Fitting 5 folds for each of 32 candidates, totalling 160 fits
[CV] END .......................C=1, gamma=1, kernel=sigmoid; total time=   0.1s
[CV] END .......................C=1, gamma=1, kernel=sigmoid; total time=   0.1s
[CV] END .......................C=1, gamma=1, kernel=sigmoid; total time=   0.1s
[CV] END .......................C=1, gamma=1, kernel=sigmoid; total time=   0.1s
[CV] END .......................C=1, gamma=1, kernel=sigmoid; total time=   0.1s
[CV] END ...........................C=1, gamma=1, kernel=rbf; total time=   0.8s
[CV] END ...........................C=1, gamma=1, kernel=rbf; total time=   0.8s
[CV] END ...........................C=1, gamma=1, kernel=rbf; total time=   0.8s
[CV] END ...........................C=1, gamma=1, kernel=rbf; total time=   0.8s
[CV] END ...........................C=1, gamma=1, kernel=rbf; total time=   0.8s
[CV] END .....................C=1, gamma=0.1, kernel=sigmoid; total time=   0.1s
[CV] END .....................C=1, gamma=0.1, k

GridSearchCV(estimator=SVC(),
             param_grid={'C': [1, 10, 100, 1000],
                         'gamma': [1, 0.1, 0.001, 0.0001],
                         'kernel': ['sigmoid', 'rbf']},
             verbose=2)

In [177]:
grid.best_params_
predic = grid.predict(X_test)
print(grid.score(X_test, y_test))
print(grid.best_params_)

0.6967741935483871
{'C': 10, 'gamma': 0.0001, 'kernel': 'rbf'}


In [178]:
results_df = save_results(clf, X_test, y_test, 'SVC', results_df)


C:\Users\polcg\.virtualenvs\MD2-NovaiZ3s\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# Random Forest

In [179]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification

In [180]:
# dataset_path = 'allUsers.lcl.csv'

df = pd.read_csv(dataset_path)
#knn Cannot work with mining data, so replace them with a 0
df = df.replace('?', 0)
#df = df.dropna(thresh=len(df) * 0.10, axis=1)


# Class recognition with User label

In [181]:
df2 = df.drop(['User'], axis=1)
labels_Users = df['User']
X_train, X_test, y_train, y_test,  = train_test_split(df2, labels_Users, test_size=0.20, random_state=42)

In [182]:
clf = RandomForestClassifier(max_depth=200, random_state=10)
clf.fit(X_train, y_train)
clf.score(X_test, y_test)

0.9863636363636363

In [183]:
results_df = save_results(clf, X_test, y_test, 'Random forest', results_df)


# Gread Search

## Show results:

In [184]:
results_df.sort_values(by=['acc'], ascending=False)


,acc,prec,recall,test f1 score (W)
Random forest,0.986364,0.986464,0.986364,0.940244
KNN,0.864661,0.863969,0.864661,0.796469
SVC,0.170968,0.726333,0.170968,0.072726
